# Import des librairies

In [1]:
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

!pip install pandas_profiling
from pandas_profiling import ProfileReport

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

!pip install plotly

import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

pio.renderers.default = "iframe"

pd.options.plotting.backend = "plotly"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.8/679.8 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 100.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 120.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 60.7 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27079 sha256=f8a9ea34b5c9a9034ad4d88f1f82884dad54bd051a31ecab2a10d098dff78f68
  Stored in directory: /root/.cache/pip/wheels/1d/05/04/c6d7d3b66539d9e659ac6dfe81e2d0fd4c1a8316cc5a403300
Successfully built htmlmin
  Attemptin

# Chargement des fichiers

In [2]:
DATA_PATH = "/storage/P9"

## Load and describe Articles Metada data

articles_metadata = pd.read_csv(
    Path(DATA_PATH, "articles_metadata.csv"),
    parse_dates=["created_at_ts"],
    date_parser=lambda x: datetime.fromtimestamp(int(x) / 1000),
    dtype={
        "article_id": "category",
        "category_id": "category",
        "publisher_id": "category",
        "words_count": "int",
    },
)

articles_metadata = articles_metadata.astype({"created_at_ts": "datetime64[ns]"})

articles_metadata.describe(include="all", datetime_is_numeric=True)

,article_id,category_id,created_at_ts,publisher_id,words_count
count,364047,364047,364047,364047,364047.000000
unique,364047,461,NaN,1,NaN
top,0,281,NaN,0,NaN
freq,1,12817,NaN,364047,NaN
mean,NaN,NaN,2016-09-16 23:57:17.328421888,NaN,190.897727
min,NaN,NaN,2006-09-27 11:14:35,NaN,0.000000
25%,NaN,NaN,2015-10-15 16:00:43.500000,NaN,159.000000
50%,NaN,NaN,2017-03-13 16:27:29,NaN,186.000000
75%,NaN,NaN,2017-11-05 14:09:11,NaN,218.000000
max,NaN,NaN,2018-03-13 12:12:30,NaN,6690.000000


In [3]:
## Loading all clicks data

clicks = pd.concat(
    [
        pd.read_csv(
            click_file_path,
            parse_dates=["session_start", "click_timestamp"],
            date_parser=lambda x: datetime.fromtimestamp(int(int(x) / 1000)),
            dtype={
                "user_id": "category",
                "session_id": "category",
                "session_size": "int",
                "click_article_id": "category",
                "click_environment": "category",
                "click_deviceGroup": "category",
                "click_os": "category",
                "click_country": "category",
                "click_region": "category",
                "click_referrer_type": "category",
            },
        ).replace(
            {
                "click_environment": {
                    "1": "1 - Facebook Instant Article",
                    "2": "2 - Mobile App",
                    "3": "3 - AMP (Accelerated Mobile Pages)",
                    "4": "4 - Web",
                },
                "click_deviceGroup": {
                    "1": "1 - Tablet",
                    "2": "2 - TV",
                    "3": "3 - Empty",
                    "4": "4 - Mobile",
                    "5": "5 - Desktop",
                },
                "click_os": {
                    "1": "1 - Other",
                    "2": "2 - iOS",
                    "3": "3 - Android",
                    "4": "4 - Windows Phone",
                    "5": "5 - Windows Mobile",
                    "6": "6 - Windows",
                    "7": "7 - Mac OS X",
                    "8": "8 - Mac OS",
                    "9": "9 - Samsung",
                    "10": "10 - FireHbbTV",
                    "11": "11 - ATV OS X",
                    "12": "12 - tvOS",
                    "13": "13 - Chrome OS",
                    "14": "14 - Debian",
                    "15": "15 - Symbian OS",
                    "16": "16 - BlackBerry OS",
                    "17": "17 - Firefox OS",
                    "18": "18 - Android",
                    "19": "19 - Brew MP",
                    "20": "20 - Chromecast",
                    "21": "21 - webOS",
                    "22": "22 - Gentoo",
                    "23": "23 - Solaris",
                },
            }
        )
        for click_file_path in tqdm(
            sorted(Path(DATA_PATH, "clicks").glob("clicks_hour_*.csv"))
        )
    ],
    sort=False,
    ignore_index=True,
    verify_integrity=True,
)

clicks = clicks.astype(
    {"session_start": "datetime64[ns]", "click_timestamp": "datetime64[ns]"}
)

clicks.describe(include="all", datetime_is_numeric=True)

100%|██████████| 385/385 [00:26<00:00, 14.69it/s]


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
count,2988181,2988181,2988181,2.988181e+06,2988181,2988181,2988181,2988181,2988181,2988181,2988181,2988181
unique,322897,1048594,NaN,NaN,46033,NaN,3,5,8,11,28,7
top,5890,1507563657895091,NaN,NaN,160974,NaN,4 - Web,1 - Tablet,17 - Firefox OS,1,25,2
freq,1232,124,NaN,NaN,37213,NaN,2904478,1823162,1738138,2852406,804985,1602601
mean,NaN,NaN,2017-10-08 14:17:08.013157120,3.901885e+00,NaN,2017-10-08 14:51:05.106516736,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,2017-10-01 02:37:03,2.000000e+00,NaN,2017-10-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2017-10-04 13:35:52,2.000000e+00,NaN,2017-10-04 14:20:52,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2017-10-08 20:09:00,3.000000e+00,NaN,2017-10-08 20:35:30,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2017-10-11 19:16:54,4.000000e+00,NaN,2017-10-11 19:43:24,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,2017-10-17 03:36:19,1.240000e+02,NaN,2017-11-13 20:04:14,NaN,NaN,NaN,NaN,NaN,NaN


# EDA

## Metadata

In [4]:
## Visualize Articles categories distribution

articles_metadata["category_id"].value_counts().plot(
    kind="bar",
    labels={
        "index": "Category ID",
        "value": "Count",
    },
    color="value",
    title="Distribution of categories",
)

In [5]:
## Visualize number of articles per category distribution

articles_metadata["category_id"].value_counts().plot(
    kind="box",
    x="category_id",
    title="Distribution of categories counts",
    labels={
        "index": "Category ID",
        "category_id": "Count",
    },
    notched=True,
    points="suspectedoutliers",
)

In [6]:
## Visualize Articles creation time distribution

articles_metadata.sample(frac=0.10)["created_at_ts"].plot(
    kind="histogram",
    title="Distribution of creation time (sampling = 10%)",
    labels={
        "value": "Creation time",
    },
    text_auto=True,
    marginal="box",
)

In [7]:
## Visualize articles word count distribution

articles_metadata["words_count"].sample(frac=0.10).plot(
    kind="histogram",
    title="Distribution of words count (sampling = 10%)",
    labels={
        "value": "Words count",
    },
    text_auto=True,
    marginal="box",
)

In [8]:
## Publish Articles Metadata ProfileReport

profile = ProfileReport(
    articles_metadata,
    title="Pandas Profiling Report",
    explorative=True,
    minimal=True,
)
profile.to_file(Path("./reports/articles_metadata_profile_report.html"))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Clicks

In [8]:
## Print period information

print('Sessions start from {} to {}'.format(clicks.session_start.min(), clicks.session_start.max()))
print('Clicks period starts from {} to {}'.format(clicks.click_timestamp.min(), clicks.click_timestamp.max()))

Sessions start from 2017-10-01 02:37:03 to 2017-10-17 03:36:19
Clicks period starts from 2017-10-01 03:00:00 to 2017-11-13 20:04:14


In [9]:
## Visualize click sessions distribution over time

clicks.sample(frac=0.10)["session_start"].plot(
    kind="histogram",
    title="Distribution of sessions (sampling = 10%)",
    labels={
        "value": "Sessions",
    },
    text_auto=True,
    marginal="box",
)

In [10]:
## Visualize clicks distribution over time

clicks.sample(frac=0.10)["click_timestamp"].plot(
    kind="histogram",
    title="Distribution of clicks (sampling = 10%)",
    labels={
        "value": "Clicks",
    },
    text_auto=True,
    marginal="box",
)

In [11]:
## Visualize clicks sessions size

clicks.sample(frac=0.10)["session_size"].plot(
    kind="histogram",
    title="Distribution of session sizes (sampling = 10%)",
    labels={
        "value": "Session size",
    },
    text_auto=True,
    marginal="box",
)

In [12]:
## Visualize nb of clicked articles per user (user engagement)

clicks.sample(frac=0.10).groupby("user_id").agg(
    COUNT_unique_article=("click_article_id", lambda x: len(set(list(x)))),
).plot(
    kind="histogram",
    title="Distribution of number of clicked articles (sampling = 10%)",
    labels={
        "value": "Number of clicked articles",
    },
    text_auto=True,
    marginal="box",
)

In [13]:
## Visualize click environment : user environment

fig = px.parallel_categories(
    clicks.sample(frac=0.10),
    dimensions=["click_environment", "click_deviceGroup", "click_os"],
    title="Distribution of Environment x Device Group x OS (sampling = 10%)",
    labels={
        "click_environment": "Environment",
        "click_deviceGroup": "Device group",
        "click_os": "OS",
    },
)
fig.show()

In [14]:
## Visualize click location : user geolocation

fig = px.parallel_categories(
    clicks.sample(frac=0.10),
    dimensions=["click_country", "click_region"],
    title="Distribution of Country x Region (sampling = 10%)",
    labels={
        "click_country": "Country",
        "click_region": "Region",
    },
)
fig.show()

In [15]:
## Visualize click referrer : user referrer

clicks.sample(frac=0.10)["click_referrer_type"].plot(
    kind="histogram",
    title="Distribution of referrer types (sampling = 10%)",
    labels={
        "value": "Referrer type",
    },
    category_orders={
        "value": [str(i) for i in range(1, 8)],
    },
    text_auto=True,
)

In [32]:
## Publish clicks metadata ProfileReport

profile = ProfileReport(
    clicks, title="Pandas Profiling Report", explorative=True, minimal=True
)
profile.to_file(Path("./reports/clicks_profile_report.html"))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Embeddings

In [19]:
## Loading articles embeddings data

articles_embeddings = pd.read_pickle(Path(DATA_PATH, "articles_embeddings.pickle"))

articles = pd.DataFrame(
    articles_embeddings,
    columns=["embedding_" + str(i) for i in range(articles_embeddings.shape[1])],
)
articles["words_count"] = articles_metadata["words_count"]
articles["category_id"] = articles_metadata["category_id"]
articles["article_id"] = articles_metadata["article_id"]

articles.describe(include="all", datetime_is_numeric=True)

articles_sample = articles.sample(frac=0.10)

In [17]:
## Visualize Articles Embeddings in 2D PCA

pca = PCA(n_components=2)
articles_pca = pca.fit_transform(
    articles_sample[
        ["embedding_" + str(i) for i in range(articles_embeddings.shape[1])]
    ]
)

# Plot the data in the PCA space
fig = px.scatter(
    x=articles_pca[:, 0],
    y=articles_pca[:, 1],
    color=articles_sample["category_id"],
    symbol=articles_sample["category_id"],
    title="PCA 2D",
    width=1200,
    height=800,
)
fig.show()

In [18]:
## Visualize Articles Embeddings in 2D t-SNE

tsne = TSNE(n_components=2)
articles_tsne = tsne.fit_transform(
    articles_sample[
        ["embedding_" + str(i) for i in range(articles_embeddings.shape[1])]
    ]
)

# Plot the data in the PCA space
fig = px.scatter(
    x=articles_tsne[:, 0],
    y=articles_tsne[:, 1],
    color=articles_sample["category_id"],
    symbol=articles_sample["category_id"],
    title="t-SNE 2D",
    width=1200,
    height=800,
)
fig.show()

/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

